In [138]:
import xarray as xr

In [836]:
import numpy as np
import pandas as pd
import xarray as xr

# Set the dimensions for x, y, and time
x_dim = 9  # Number of elements along the x dimension
y_dim = 11  # Number of elements along the y dimension
time_dim = pd.date_range(
    start="2020-01-01", end="2024-01-01", freq="ME"
)  # Monthly frequency

np.random.seed(seed=42)

# Generate random data
data = np.random.random(size=(len(time_dim), y_dim, x_dim)) * 100

# Create the xarray DataArray
data = xr.DataArray(
    data,
    dims=["time", "y", "x"],
    coords={"time": time_dim, "x": np.arange(x_dim), "y": np.arange(y_dim)},
)

In [860]:
path = (
    "C:\\Users\\ospin\\OneDrive\\Documents\\CMP\\Delete_ask_julianDelete_ask_julian\\"
)

In [862]:
xr.open_zarr(path + "ScenarioMIP.NOAA-GFDL.GFDL-CM4.ssp585.r1i1p1f1.day.gr2.zarr")

<xarray.Dataset> Size: 8MB
Dimensions:     (time: 31390, plev: 8, y: 3, x: 2, bnds: 2)
Coordinates:
  * x           (x) float64 16B 286.2 288.8
Dimensions without coordinates: time, plev, y, bnds
Data variables:
    hus         (time, plev, y, x) float32 6MB dask.array<chunksize=(200, 8, 3, 2), meta=np.ndarray>
    lat_bounds  (y, bnds) float64 48B dask.array<chunksize=(3, 2), meta=np.ndarray>
    pr          (time, y, x) float64 2MB dask.array<chunksize=(600, 3, 2), meta=np.ndarray>
    tas         (time, y, x) float32 753kB dask.array<chunksize=(1575, 3, 2), meta=np.ndarray>
Attributes: (12/49)
    Conventions:            CF-1.7 CMIP-6.0 UGRID-1.0
    activity_id:            ScenarioMIP
    branch_method:          standard
    branch_time_in_child:   0.0
    branch_time_in_parent:  60225.0
    comment:                <null ref>
    ...                     ...
    title:                  NOAA GFDL GFDL-CM4 model output prepared for CMIP...
    tracking_id:            hdl:21.14100/19a7361f-a1f3-42a4-b93a-113d5961205a...
    variable_id:            hus
    variant_info:           N/A
    variant_label:          r1i1p1f1
    version_id:             v20180701

In [840]:
def spatial_storm_aggregation(da, center_x, center_y, radius):
    """ """
    xx, yy = np.meshgrid(da.coords["x"].values, da.coords["y"].values)
    coor_x = xx.flatten()
    coor_y = yy.flatten()

    sq_dist = (coor_x - center_x) ** 2 + (coor_y - center_y) ** 2
    within_r_ind = np.where(sq_dist <= radius**2)

    within_r_x = coor_x[within_r_ind]
    within_r_y = coor_y[within_r_ind]

    da_cut = da.where((da.x.isin(within_r_x)) & (da.y.isin(within_r_y)), drop=True)

    dict_config = {  # y, x
        "1": [[0], [0]],
        "2": [[0, 0], [0, 1]],
        "3": [[0, 0, -1], [0, 1, 0]],
        "4": [[0, 0, -1, -1], [0, 1, 0, 1]],
        "5.a": [[0, 0, -1, -1, 0], [0, 1, 0, 1, -1]],
        "5.b": [[0, 0, 0, 1, 1], [0, 1, -1, 0, 1]],
        "6": [[0, 0, 0, 1, 1, 1], [0, 1, -1, 0, 1, -1]],
        "7": [[0, 0, 0, -1, -1, -1, 1], [0, 1, -1, 0, 1, -1, 0]],
        "8": [[0, 0, 0, -1, -1, -1, 1, 1], [0, 1, -1, 0, 1, -1, 0, 1]],
        "9": [[0, 0, 0, -1, -1, -1, 1, 1, 1], [0, 1, -1, 0, 1, -1, 0, 1, -1]],
        "10.a": [[0, 0, 0, 0, -1, -1, -1, -1, 1, 1], [-1, 0, 1, 2, -1, 0, 1, 2, 0, 1]],
        "10.b": [
            [0, 0, 0, 0, -1, -1, -1, -1, 1, 1],
            [-2, -1, 0, 1, -2, -1, 0, 1, -1, 0],
        ],
        "12": [
            [0, 0, 0, 0, -1, -1, -1, -1, 1, 1, 1, 1],
            [-1, 0, 1, 2, -1, 0, 1, 2, -1, 0, 1, 2],
        ],
        "14.a": [
            [0, 0, 0, 0, -1, -1, -1, -1, 1, 1, 1, 1, 2, 2],
            [-1, 0, 1, 2, -1, 0, 1, 2, -1, 0, 1, 2, 0, 1],
        ],
        "14.b.": [
            [0, 0, 0, 0, -1, -1, -1, -1, 1, 1, 1, 1, 2, 2],
            [-2, -1, 0, 1, -2, -1, 0, 1, -2, -1, 0, 1, -1, 0],
        ],
        "16": [
            [0, 0, 0, 0, -1, -1, -1, -1, 1, 1, 1, 1, 2, 2, 2, 2],
            [-1, 0, 1, 2, -1, 0, 1, 2, -1, 0, 1, 2, -1, 0, 1, 2],
        ],
        "18.a": [
            [0, 0, 0, 0, -2, -2, -2, -2, -1, -1, -1, -1, 1, 1, 1, 1, 2, 2],
            [-1, 0, 1, 2, -1, 0, 1, 2, -1, 0, 1, 2, -1, 0, 1, 2, 0, 1],
        ],
        "18.b": [
            [0, 0, 0, 0, -2, -2, -2, -2, -1, -1, -1, -1, 1, 1, 1, 1, 2, 2],
            [-2, -1, 0, 1, -2, -1, 0, 1, -2, -1, 0, 1, -2, -1, 0, 1, -1, 0],
        ],
        "20": [
            [0, 0, 0, 0, -2, -2, -2, -2, -1, -1, -1, -1, 1, 1, 1, 1, 2, 2, 2, 2],
            [-1, 0, 1, 2, -1, 0, 1, 2, -1, 0, 1, 2, -1, 0, 1, 2, -1, 0, 1, 2],
        ],
        "23": [
            [
                -2,
                -2,
                -2,
                -2,
                -2,
                -1,
                -1,
                -1,
                -1,
                -1,
                0,
                0,
                0,
                0,
                0,
                1,
                1,
                1,
                1,
                1,
                2,
                2,
                2,
            ],
            [
                -2,
                -1,
                0,
                1,
                2,
                -2,
                -1,
                0,
                1,
                2,
                -2,
                -1,
                0,
                1,
                2,
                -2,
                -1,
                0,
                1,
                2,
                -1,
                0,
                1,
            ],
        ],
        "24.a": [
            [
                -2,
                -2,
                -2,
                -2,
                -1,
                -1,
                -1,
                -1,
                0,
                0,
                0,
                0,
                1,
                1,
                1,
                1,
                2,
                2,
                2,
                2,
                3,
                3,
                3,
                3,
            ],
            [
                -1,
                0,
                1,
                2,
                -1,
                0,
                1,
                2,
                -1,
                0,
                1,
                2,
                -1,
                0,
                1,
                2,
                -1,
                0,
                1,
                2,
                -1,
                0,
                1,
                2,
            ],
        ],
        "24.b": [
            [
                -2,
                -2,
                -2,
                -2,
                -1,
                -1,
                -1,
                -1,
                0,
                0,
                0,
                0,
                1,
                1,
                1,
                1,
                2,
                2,
                2,
                2,
                3,
                3,
                3,
                3,
            ],
            [
                -2,
                -1,
                0,
                1,
                -2,
                -1,
                0,
                1,
                -2,
                -1,
                0,
                1,
                -2,
                -1,
                0,
                1,
                -2,
                -1,
                0,
                1,
                -2,
                -1,
                0,
                1,
            ],
        ],
        "25": [
            [
                -2,
                -2,
                -2,
                -2,
                -2,
                -1,
                -1,
                -1,
                -1,
                -1,
                0,
                0,
                0,
                0,
                0,
                1,
                1,
                1,
                1,
                1,
                2,
                2,
                2,
                2,
                2,
            ],
            [
                -2,
                -1,
                0,
                1,
                2,
                -2,
                -1,
                0,
                1,
                2,
                -2,
                -1,
                0,
                1,
                2,
                -2,
                -1,
                0,
                1,
                2,
                -2,
                -1,
                0,
                1,
                2,
            ],
        ],
    }

    da_cut_c = da_cut[0, :, :].copy(deep=True) * np.nan

    da_for_coor = da_cut_c.where(
        ~((da_cut_c.x == center_x) & (da_cut_c.y == center_y)), 1
    )
    centers_pos = np.argwhere(da_for_coor.values == 1)

    ds_mean = []
    for name, confi in dict_config.items():

        cent_pos_y = confi[0] + centers_pos[0][0]
        cent_pos_x = confi[1] + centers_pos[0][1]

        matrix_mask = da_cut_c.values.copy()

        try:
            matrix_mask[cent_pos_y, cent_pos_x] = 1
        except:
            continue

        for rot in range(4):
            matrix_mask_rot = np.rot90(matrix_mask, k=rot)
            da_pr = da_cut * matrix_mask_rot
            ds_mean.append(
                da_pr.mean(dim=["x", "y"]).expand_dims(
                    dim={"conf": [name + "_" + str(rot)]}
                )
            )

            if np.array_equal(np.rot90(matrix_mask, k=1), matrix_mask, equal_nan=True):
                break

    return xr.concat(ds_mean, dim="time")

In [841]:
def spatial_average_storm_configurations(da, radius):
    """Compute the spatial average for different storm configurations proposed by Clavet-Gaumont et al. (2017).

    Parameters
    ----------
    da : xr.DataArray
        DataArray containing the precipitation values.
    radius : float
        Maximum radius of the storm.

    Returns
    -------
    xr.DataSet
        DataSet contaning the spatial averages for all the storm configurations.

    Notes
    -----
    https://doi.org/10.1016/j.ejrh.2017.07.003.
    """
    dict_config = {
        "1": [[0], [0]],
        "2.1": [[0, 0], [0, 1]],
        "2.2": [[0, 1], [0, 0]],
        "3.1": [[0, 1, 1], [0, 0, 1]],
        "3.2": [[1, 1, 0], [0, 1, 1]],
        "3.3": [[0, 0, 1], [0, 1, 1]],
        "3.4": [[0, 1, 0], [0, 0, 1]],
        "4.1": [[0, 0, 1, 1], [0, 1, 0, 1]],
        "5.1": [[0, 0, 1, 1, 1], [1, 2, 0, 1, 2]],
        "5.2": [[0, 0, 0, 1, 1], [0, 1, 2, 1, 2]],
        "5.3": [[0, 0, 0, 1, 1], [0, 1, 2, 0, 1]],
        "5.4": [[0, 0, 1, 1, 1], [0, 1, 0, 1, 2]],
        "5.5": [[0, 0, 1, 1, 2], [0, 1, 0, 1, 0]],
        "5.6": [[0, 0, 1, 1, 2], [0, 1, 0, 1, 1]],
        "5.7": [[0, 1, 1, 2, 2], [0, 0, 1, 0, 1]],
        "5.8": [[0, 1, 1, 2, 2], [1, 0, 1, 0, 1]],
        "6.1": [[0, 0, 0, 1, 1, 1], [0, 1, 2, 0, 1, 2]],
        "6.2": [[0, 0, 1, 1, 2, 2], [0, 1, 0, 1, 0, 1]],
        "7.1": [[0, 0, 0, 1, 1, 1, 2], [0, 1, 2, 0, 1, 2, 1]],
        "7.2": [[0, 0, 1, 1, 1, 2, 2], [0, 1, 0, 1, 2, 0, 1]],
        "7.3": [[0, 1, 1, 1, 2, 2, 2], [1, 0, 1, 2, 0, 1, 2]],
        "7.4": [[0, 0, 1, 1, 1, 2, 2], [1, 2, 0, 1, 2, 1, 2]],
        "8.1": [[0, 0, 0, 1, 1, 1, 2, 2], [0, 1, 2, 0, 1, 2, 1, 2]],
        "8.2": [[0, 0, 1, 1, 1, 2, 2, 2], [0, 1, 0, 1, 2, 0, 1, 2]],
        "8.3": [[0, 0, 1, 1, 1, 2, 2, 2], [1, 2, 0, 1, 2, 0, 1, 2]],
        "8.4": [[0, 0, 0, 1, 1, 1, 2, 2], [0, 1, 2, 0, 1, 2, 0, 1]],
        "9.1": [[0, 0, 0, 1, 1, 1, 2, 2, 2], [0, 1, 2, 0, 1, 2, 0, 1, 2]],
        "10.1": [[0, 0, 0, 0, 1, 1, 1, 1, 2, 2], [0, 1, 2, 3, 0, 1, 2, 3, 1, 2]],
        "10.2": [[0, 0, 1, 1, 1, 1, 2, 2, 2, 2], [1, 2, 0, 1, 2, 3, 0, 1, 2, 3]],
        "10.3": [[0, 0, 1, 1, 1, 2, 2, 2, 3, 3], [0, 1, 0, 1, 2, 0, 1, 2, 0, 1]],
        "10.4": [[0, 0, 1, 1, 1, 2, 2, 2, 3, 3], [1, 2, 0, 1, 2, 0, 1, 2, 1, 2]],
        "12.1": [
            [0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2],
            [0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3],
        ],
        "12.2": [
            [0, 0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3],
            [0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2],
        ],
        "14.1": [
            [0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3],
            [0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 1, 2],
        ],
        "14.2": [
            [0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3],
            [0, 1, 2, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2],
        ],
        "14.3": [
            [0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3],
            [1, 2, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3],
        ],
        "14.4": [
            [0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3],
            [1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 1, 2, 3],
        ],
        "16.1": [
            [0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3],
            [0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3],
        ],
        "18.1": [
            [0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3],
            [0, 1, 2, 3, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3],
        ],
        "18.2": [
            [0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3],
            [1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 1, 2, 3, 4],
        ],
        "18.3": [
            [0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4],
            [0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 1, 2],
        ],
        "18.4": [
            [0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4],
            [1, 2, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3],
        ],
        "20.1": [
            [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3],
            [0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4],
        ],
        "20.2": [
            [0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4],
            [0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3],
        ],
        "23.1": [
            [0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 4, 4, 4, 4],
            [0, 1, 2, 3, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3],
        ],
        "23.2": [
            [0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 4, 4, 4, 4],
            [1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 1, 2, 3, 4],
        ],
        "23.3": [
            [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 4, 4, 4],
            [0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 1, 2, 3],
        ],
        "23.4": [
            [0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4],
            [1, 2, 3, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4],
        ],
        "24.1": [
            [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3],
            [0, 1, 2, 3, 4, 5, 0, 1, 2, 3, 4, 5, 0, 1, 2, 3, 4, 5, 0, 1, 2, 3, 4, 5],
        ],
        "24.2": [
            [0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4, 5, 5, 5, 5],
            [0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3],
        ],
        "25.1": [
            [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4],
            [0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4],
        ],
    }
    x = da.cf.axes["X"][0]
    y = da.cf.axes["Y"][0]

    # Pixel size
    dy = (da[y][1] - da[y][0]).values
    dx = (da[x][1] - da[x][0]).values

    # Number of pixels in da
    npy_da = len(da[y])
    npx_da = len(da[x])

    spt_av_list = []
    for name, confi in dict_config.items():

        conf_y = confi[0]
        conf_x = confi[1]

        # Number of pixels in the configuration
        npy_confi = np.abs(conf_y).max() + 1
        npx_confi = np.abs(conf_x).max() + 1

        # Checks that the configuration size is within the desired storn size.
        if (dy * npy_confi / 2 > radius) or (dx * npx_confi / 2 > radius):
            break

        # Checks that the configuration fits in the domain.
        if (npy_confi > npy_da) or (npx_confi > npx_da):
            break

        # Number of times a configuration can be shifted in each axis
        ny = len(da[y]) - npy_confi + 1
        nx = len(da[x]) - npx_confi + 1

        # List with the configuration duplicated as many times as indicated by nx and nx.
        conf_y_ex = np.reshape(np.array(conf_y * ny), (ny, len(conf_y)))
        conf_x_ex = np.reshape(np.array(conf_x * nx), (nx, len(conf_x)))

        # List with the incrementes from 0 to nx and ny
        inc_y = np.ones(conf_y_ex.shape) * [[i] for i in range(ny)]
        inc_x = np.ones(conf_x_ex.shape) * [[i] for i in range(nx)]

        # Shifted configurations
        pos_y = (conf_y_ex + inc_y).astype(int)
        pos_x = (conf_x_ex + inc_x).astype(int)

        # List of all the combinations of the shifted configurations
        shifted_confi = list(product(pos_y, pos_x))

        for shift, confi_shifted in enumerate(shifted_confi):
            matrix_mask = np.full((len(da[y]), len(da[x])), np.nan)
            matrix_mask[(confi_shifted[0], confi_shifted[1])] = 1
            da_mask = da * matrix_mask
            da_mean = da_mask.mean(dim=[x, y]).expand_dims(
                dim={"conf": [name + "_" + str(shift)]}
            )

            spt_av_list.append(da_mean)

    spt_av = xr.concat(spt_av_list, dim="conf")

    if "units" in da.attrs:
        spt_av.attrs["units"] = da.attrs["units"]

    return spt_av

In [848]:
da = data
radius = 100000000

dict_config = {  # y, x
    # "1": [[0], [0]],
    # "2": [[0, 0], [0, 1]],
    # "3": [[0, 0, -1], [0, 1, 0]],
    # "4": [[0, 0, -1, -1], [0, 1, 0, 1]],
    # "5.a": [[0, 0, -1, -1, 0], [0, 1, 0, 1, -1]],
    # "5.b": [[0, 0, -1, -1, -1], [0, 1, -1, 0, 1]],
    # "6": [[0, 0, 0, 1, 1, 1], [0, 1, -1, 0, 1, -1]],
    # "7": [[0, 0, 0, -1, -1, -1, 1], [0, 1, -1, 0, 1, -1, 0]],
    # "8": [[0, 0, 0, -1, -1, -1, 1, 1], [0, 1, -1, 0, 1, -1, 0, 1]],
    # "9": [[0, 0, 0, -1, -1, -1, 1, 1, 1], [0, 1, -1, 0, 1, -1, 0, 1, -1]],
    # "10": [[0, 0, 0, 0, -1, -1, -1, -1, 1, 1], [-1, 0, 1, 2, -1, 0, 1, 2, 0, 1]],
    # "12": [
    #     [0, 0, 0, 0, -1, -1, -1, -1, 1, 1, 1, 1],
    #     [-1, 0, 1, 2, -1, 0, 1, 2, -1, 0, 1, 2],
    # ],
    # "14": [
    #     [0, 0, 0, 0, -1, -1, -1, -1, 1, 1, 1, 1, 2, 2],
    #     [-1, 0, 1, 2, -1, 0, 1, 2, -1, 0, 1, 2, 0, 1],
    # ],
    # "16": [
    #     [0, 0, 0, 0, -1, -1, -1, -1, 1, 1, 1, 1, 2, 2, 2, 2],
    #     [-1, 0, 1, 2, -1, 0, 1, 2, -1, 0, 1, 2, -1, 0, 1, 2],
    # ],
    # "18": [
    #     [0, 0, 0, 0, -2, -2, -2, -2, -1, -1, -1, -1, 1, 1, 1, 1, 2, 2],
    #     [-1, 0, 1, 2, -1, 0, 1, 2, -1, 0, 1, 2, -1, 0, 1, 2, 0, 1],
    # ],
    "20": [
        [0, 0, 0, 0, -2, -2, -2, -2, -1, -1, -1, -1, 1, 1, 1, 1, 2, 2, 2, 2],
        [-1, 0, 1, 2, -1, 0, 1, 2, -1, 0, 1, 2, -1, 0, 1, 2, -1, 0, 1, 2],
    ],
    # '23': [[-2,-2,-2,-2,-2,-1,-1,-1,-1,-1,0,0,0,0,0,1,1,1,1,1,2,2,2],
    #         [-2,-1,0,1,2,-2,-1,0,1,2,-2,-1,0,1,2,-2,-1,0,1,2,-1,0,1]
    # ],
    # '24': [[-2,-2,-2,-2,-1,-1,-1,-1,0,0,0,0,1,1,1,1,2,2,2,2,3,3,3,3],
    #             [-1,0,1,2,-1,0,1,2,-1,0,1,2,-1,0,1,2,-1,0,1,2,-1,0,1,2]
    # ],
    # '25': [[-2,-2,-2,-2,-2,-1,-1,-1,-1,-1,0,0,0,0,0,1,1,1,1,1,2,2,2,2,2],
    #         [-2,-1,0,1,2,-2,-1,0,1,2,-2,-1,0,1,2,-2,-1,0,1,2,-2,-1,0,1,2]
    # ],
}

In [849]:
da

<xarray.DataArray (time: 48, y: 11, x: 9)> Size: 38kB
array([[[3.74540119e+01, 9.50714306e+01, 7.31993942e+01, ...,
         5.80836122e+00, 8.66176146e+01, 6.01115012e+01],
        [7.08072578e+01, 2.05844943e+00, 9.69909852e+01, ...,
         1.83404510e+01, 3.04242243e+01, 5.24756432e+01],
        [4.31945019e+01, 2.91229140e+01, 6.11852895e+01, ...,
         4.56069984e+01, 7.85175961e+01, 1.99673782e+01],
        ...,
        [5.52211712e-01, 8.15461428e+01, 7.06857344e+01, ...,
         3.58465729e+01, 1.15869060e+01, 8.63103426e+01],
        [6.23298127e+01, 3.30898025e+01, 6.35583503e+00, ...,
         6.37557471e+01, 8.87212743e+01, 4.72214925e+01],
        [1.19594246e+01, 7.13244787e+01, 7.60785049e+01, ...,
         5.22732829e+01, 4.27541018e+01, 2.54191267e+00]],

       [[1.07891427e+01, 3.14291857e+00, 6.36410411e+01, ...,
         2.49292229e+01, 4.10382923e+01, 7.55551139e+01],
        [2.28798165e+01, 7.69799098e+00, 2.89751453e+01, ...,
         6.33403757e+01, 8.71460590e+01, 8.03672077e+01],
        [1.86570059e+01, 8.92558998e+01, 5.39342242e+01, ...,
         1.10051925e+01, 2.27935163e+01, 4.27107789e+01],
...
        [3.31616849e+01, 5.00579317e+01, 6.45725043e+01, ...,
         8.02396031e+01, 8.61418046e+01, 1.37300033e+01],
        [9.52386752e+01, 3.28822618e+01, 6.62313739e+01, ...,
         3.12919548e+01, 8.58506455e+01, 1.31686903e+01],
        [7.05040906e+01, 7.41977730e+01, 6.77723241e+01, ...,
         5.36347915e+00, 7.17464991e+01, 4.75475715e+01]],

       [[8.49098020e+01, 3.83397141e+01, 1.68278589e+01, ...,
         7.20383999e+01, 5.11113638e+01, 6.04312982e+01],
        [4.60898338e+01, 8.28708984e+01, 8.29562716e+01, ...,
         6.71012981e+01, 6.97024166e+01, 3.62982840e+01],
        [8.79279556e+01, 7.25371600e+01, 1.12590467e+01, ...,
         9.61416620e+01, 6.79162568e+01, 6.31375978e+01],
        ...,
        [2.27038218e-02, 9.15001538e+01, 8.43879411e+01, ...,
         6.76047495e+00, 7.24029189e+01, 9.28807961e+01],
        [4.30535821e+01, 1.19140672e+01, 5.12392821e+01, ...,
         7.18718113e+01, 1.10603599e+01, 3.55902308e+00],
        [9.94330303e+01, 2.32728349e+01, 5.38305249e+00, ...,
         8.02401784e+01, 3.41244450e+01, 4.44183407e+01]]])
Coordinates:
  * time     (time) datetime64[ns] 384B 2020-01-31 2020-02-29 ... 2023-12-31
  * x        (x) int32 36B 0 1 2 3 4 5 6 7 8
  * y        (y) int32 44B 0 1 2 3 4 5 6 7 8 9 10

In [850]:
from itertools import product

import matplotlib.pyplot as plt

path_fig = "C:\\Users\\ospin\\OneDrive\\Documents\\CMP\\Figures\\"

In [851]:
confi_ds = xr.Dataset()

In [859]:
x = da.x[0]  ##############################
y = da.y[0]

# Pixel size
dy = (da.y[1] - da.y[0]).values  ##############################
dx = (da.x[1] - da.x[0]).values

# Number of pixels in da
npy_da = len(da.x)  ##############################
npx_da = len(da.y)

confi_ds = xr.Dataset()
da_stacked = da.stack(stacked_coords=("y", "x"))
for name, confi in dict_config.items():

    conf_y = confi[0]
    conf_x = confi[1]

    # Number of pixels in the configuration
    npy_confi = np.abs(conf_y).max() + np.abs(conf_y).min() + 1
    npx_confi = np.abs(conf_x).max() + np.abs(conf_x).min() + 1

    # Checks that the configuration size is within the desired storn size.
    if (dy * npy_confi / 2 > radius) or (dx * npx_confi / 2 > radius):
        break

    # Checks that the configuration fits in the domain.
    if (npy_confi > npy_da) or (npx_confi > npx_da):
        break

    # Number of times a configuration can be shifted in each axis
    ny = len(da.y) - npy_confi + 1  ################################
    nx = len(da.x) - npx_confi + 1

    # List with the configuration duplicated as many times as indicated by nx and nx.
    conf_y_ex = np.reshape(np.array(conf_y * ny), (ny, len(conf_y)))
    conf_x_ex = np.reshape(np.array(conf_x * nx), (nx, len(conf_x)))

    # List with the incrementes from 0 to nx and ny
    inc_y = np.ones(conf_y_ex.shape) * [[i] for i in range(ny)]
    inc_x = np.ones(conf_x_ex.shape) * [[i] for i in range(nx)]

    # Shifted configurations
    pos_y = (conf_y_ex + inc_y).astype(int)
    pos_x = (conf_x_ex + inc_x).astype(int)

    # Drop shifted configuration that are out of the domain
    pos_y = pos_y[~np.any(pos_y < 0, axis=(1))]
    pos_x = pos_x[~np.any(pos_x < 0, axis=(1))]

    # List of all the combinations of the shifted configurations
    shifted_confi = list(product(pos_y, pos_x))

    # Center of the configuration
    cen_conf = np.where((np.array(conf_y) == 0) & (np.array(conf_x) == 0))[0][0]

    list_conf = []
    for shift, confi_shifted in enumerate(shifted_confi):
        matrix_mask = np.full(da.isel(time=0).shape, np.nan)  #####################

        shift_y = confi_shifted[0]
        shift_x = confi_shifted[1]

        matrix_mask[(shift_y, shift_x)] = 1

        cen_shift_y = shift_y[cen_conf]
        cen_shift_x = shift_x[cen_conf]

        list_rot = []
        for rot in range(4):
            matrix_mask_rot = _rotate_around_center(
                matrix_mask, (cen_shift_y, cen_shift_x), k=rot
            )

            if np.sum(~np.isnan(matrix_mask_rot)) != np.shape(
                confi_shifted[0]
            ):  # Include nan to mask the cases when the configuration is out of the domain.
                matrix_mask_rot[:, :] = np.nan

            da_mask = da * matrix_mask_rot

            list_rot.append(
                da_mask.mean(dim=["x", "y"]).expand_dims(
                    dim={"conf": [name + "_" + str(rot)]}
                )
            )
            da_mask[0, :, :].plot(vmin=0, vmax=100)
            plt.savefig(path_fig + name + "_" + str(shift) + "_" + str(rot) + ".png")
            plt.close("all")
        rot_concat = (
            xr.concat(list_rot, dim="conf")
            .max(dim="conf", skipna=False)
            .expand_dims(
                dim={"conf": [name], "y": [da.y[cen_shift_y]], "x": [da.x[cen_shift_x]]}
            )
            .stack(stacked_coords=("y", "x"))
            .drop_vars(["conf"])
        )

        list_conf.append(rot_concat)

    confi_ds[name] = xr.concat(list_conf, dim="stacked_coords").reindex_like(
        da_stacked, fill_value=np.nan
    )

confi_ds = confi_ds.squeeze().unstack("stacked_coords")

if "units" in da.attrs:
    confi_ds.attrs["units"] = da.attrs["units"]

In [858]:
matrix_mask

array([[nan, nan, nan, nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan,  1.,  1.,  1.,  1.],
       [nan, nan, nan, nan, nan,  1.,  1.,  1.,  1.],
       [nan, nan, nan, nan, nan,  1.,  1.,  1.,  1.],
       [nan, nan, nan, nan, nan,  1.,  1.,  1.,  1.],
       [nan, nan, nan, nan, nan,  1.,  1.,  1.,  1.]])

In [404]:
def _rotate_around_center(matrix, center, k=1):
    """
    Rotate a matrix around a given center point by 90 degrees k times.

    Parameters:
    - matrix: 2D numpy array to be rotated.
    - center: Tuple (row, col) indicating the center of rotation.
    - k: Number of 90-degree clockwise rotations.

    Returns:
    - Rotated 2D numpy array.
    """

    def _rotate_point_n_times(py, px, cy, cx, n):
        """
        Rotate a point (px, py) by 90 degrees counterclockwise around a center point (cx, cy)
        for n times.

        Args:
        px (float): The x-coordinate of the point to be rotated.
        py (float): The y-coordinate of the point to be rotated.
        cx (float): The x-coordinate of the center of rotation.
        cy (float): The y-coordinate of the center of rotation.
        n (int): The number of 90-degree rotations.

        Returns:
        tuple: The new coordinates of the point after rotation.
        """
        # Normalize n to be within [0, 3] to avoid unnecessary rotations
        n = n % 4

        # Translate point to origin (subtract the center coordinates)
        translated_x = px - cx
        translated_y = py - cy

        # Rotate point n times
        for _ in range(n):
            translated_x, translated_y = -translated_y, translated_x

        # Translate point back to original coordinates
        final_x = translated_x + cx
        final_y = translated_y + cy

        return (final_y, final_x)

    matrix = np.array(matrix)
    rows, cols = matrix.shape
    rotated_matrix = np.full_like(matrix, np.nan)

    center_row, center_col = center

    for r in range(rows):
        for c in range(cols):
            if not np.isnan(matrix[r, c]):
                new_r, new_c = _rotate_point_n_times(r, c, center_row, center_col, k)
                if 0 <= new_r < rows and 0 <= new_c < cols:
                    rotated_matrix[new_r, new_c] = matrix[r, c]

    return rotated_matrix

In [857]:
"""Compute the spatial average for different storm configurations proposed by Clavet-Gaumont et al. (2017).

Parameters
----------
da : xr.DataArray
    DataArray containing the precipitation values.
radius : float
    Maximum radius of the storm.

Returns
-------
xr.DataSet
    DataSet contaning the spatial averages for all the storm configurations.

Notes
-----
https://doi.org/10.1016/j.ejrh.2017.07.003.
"""

'Compute the spatial average for different storm configurations proposed by Clavet-Gaumont et al. (2017).\n\nParameters\n----------\nda : xr.DataArray\n    DataArray containing the precipitation values.\nradius : float\n    Maximum radius of the storm.\n\nReturns\n-------\nxr.DataSet\n    DataSet contaning the spatial averages for all the storm configurations.\n\nNotes\n-----\nhttps://doi.org/10.1016/j.ejrh.2017.07.003.\n'